In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Create an API Client 
from anthropic import Anthropic
client = Anthropic()
model = "claude-sonnet-4-0"

In [4]:
# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [5]:
def run_prompt(test_case):
    """Merges the prompt and test case input, then returns the result"""
    prompt = f"""
Please solve the following task:

{test_case["task"]}
"""
    
    messages = []
    add_user_message(messages, prompt)
    output = chat(messages)
    return output

In [6]:
def run_test_case(test_case):
    """Calls run_prompt, then grades the result"""
    output = run_prompt(test_case)
    
    # TODO - Grading
    score = 10
    
    return {
        "output": output,
        "test_case": test_case,
        "score": score
    }

In [7]:
def run_eval(dataset):
    """Loads the dataset and calls run_test_case with each case"""
    results = []
    
    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)
    
    return results

In [9]:
import json
with open("test_data.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)

In [10]:
results

[{'output': 'Here\'s a Python function to extract the AWS region from an ARN:\n\n```python\ndef extract_region_from_arn(arn):\n    """\n    Extract the AWS region from an Amazon Resource Name (ARN).\n    \n    Args:\n        arn (str): The Amazon Resource Name\n        \n    Returns:\n        str: The AWS region, or None if not found or invalid ARN\n        \n    Examples:\n        >>> extract_region_from_arn("arn:aws:s3:us-west-2:123456789012:bucket/my-bucket")\n        \'us-west-2\'\n        >>> extract_region_from_arn("arn:aws:iam::123456789012:user/username")\n        \'\'\n        >>> extract_region_from_arn("invalid-arn")\n        None\n    """\n    if not isinstance(arn, str):\n        return None\n    \n    # ARN format: arn:partition:service:region:account-id:resource\n    arn_parts = arn.split(\':\')\n    \n    # ARN must have at least 6 parts\n    if len(arn_parts) < 6:\n        return None\n    \n    # Check if it starts with \'arn\'\n    if arn_parts[0] != \'arn\':\n      